In [51]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import requests as req
from census import Census
import json

# Census API Key
#from config import api_key
c = Census("49f2db3e7faca8ecd92103d5bd5c5c765442d598", year=2016)


In [72]:
# Logic:
#Get the cites to query
#Get the census fields to query

sourceFile = pd.ExcelFile('Project1_AmazonSites.xlsx')

SitesDF=sourceFile.parse('AmazonSites') 

CitiesDf=sourceFile.parse('AmazonCities') 
CitiesDf=CitiesDf[['PlaceCode','StateCode']]

CensusFieldsDF=sourceFile.parse('CensusFields')
CensusFieldsDF = CensusFieldsDF[["Datafield","CensusCode"]]

#Converting the CensusFieldsDF into a dictionary.
dictCensusFields = CensusFieldsDF.set_index('CensusCode').to_dict()
dictCensusFields = dictCensusFields['Datafield']

print(dictCensusFields)
CitiesDf.head()

{'B01002_001E': 'Median Age', 'B25077_001E': 'Median Home Value', 'B25064_001E': 'Median Rent', 'B23025_002E': 'LaborForce', 'B23025_004E': 'Emp_Total', 'B23025_003E': 'Emp_civilian_labor_force', 'C24010_041E': 'Emp_Mgment_Occp_Female', 'C24010_005E': 'Emp_Mgment_Occp_Male', 'B15003_021E': 'Ed_Associates', 'B15003_022E': 'Ed_Bachelors', 'B15003_025E': 'Ed_Doctorate', 'B15003_018E': 'Ed_GED', 'B15003_017E': 'Ed_HighSchool', 'B15003_023E': 'Ed_Masters', 'B15003_024E': 'Ed_Professional', 'B01003_001E': 'Total Population', 'B02001_004E': 'Pop_American_Indian', 'B02001_005E': 'Population_Asian', 'B02001_003E': 'Pop_Black', 'B03001_003E': 'Pop_Hispanic_Origin', 'B02001_006E': 'Pop_Native_Hawaiian', 'B02001_007E': 'Pop_Other', 'B02001_008E': 'Pop_two_or_more_races', 'B02001_002E': 'Pop_White', 'B08136_001E': 'Commute_Time', 'B08136_004E': 'Commute_Time_Carpool', 'B08136_012E': 'Commute_Time_Other', 'B08136_007E': 'Commute_Time_Public_Transport', 'B08136_003E': 'Commute_Time_Solo_Automobile'}


,PlaceCode,StateCode
0,50000,11
1,51000,36
2,7000,25
3,14000,17
4,44000,6


In [71]:

# Build your Census API call.
arrCensusFields = ['NAME']

for k in dictCensusFields.keys():
    arrCensusFields.append(k)
    
print(arrCensusFields)
census_data= []

for city in CitiesDf.values:
    varGeo = {'for':'place:'+str(city[0]),'in':'state:'+str(city[1])}
    #census_data = c.acs5.get(arrCensusFields, varGeo)
    #print('place:'+str(city[0]))
    #print('state:'+ str(city[1]))
    temp1 = (c.acs5.get(arrCensusFields, {'for':'place:'+str(city[0]),'in':'state:'+str(city[1])}))
    print(temp1[0])
    census_data.append(temp1)
        
#print(census_data)
# Convert to DataFrame
census_pd = pd.DataFrame(census_data)

census_pd = census_pd.rename(columns=dictCensusFields)


#census_pd.head()

['NAME', 'B01002_001E', 'B25077_001E', 'B25064_001E', 'B23025_002E', 'B23025_004E', 'B23025_003E', 'C24010_041E', 'C24010_005E', 'B15003_021E', 'B15003_022E', 'B15003_025E', 'B15003_018E', 'B15003_017E', 'B15003_023E', 'B15003_024E', 'B01003_001E', 'B02001_004E', 'B02001_005E', 'B02001_003E', 'B03001_003E', 'B02001_006E', 'B02001_007E', 'B02001_008E', 'B02001_002E', 'B08136_001E', 'B08136_004E', 'B08136_012E', 'B08136_007E', 'B08136_003E']
{'NAME': 'Washington city, District of Columbia', 'B01002_001E': 33.8, 'B25077_001E': 506100.0, 'B25064_001E': 1362.0, 'B23025_002E': 385056.0, 'B23025_004E': 348225.0, 'B23025_003E': 381551.0, 'C24010_041E': 26678.0, 'C24010_005E': 29460.0, 'B15003_021E': 14496.0, 'B15003_022E': 108252.0, 'B15003_025E': 19162.0, 'B15003_018E': 12052.0, 'B15003_017E': 71315.0, 'B15003_023E': 90366.0, 'B15003_024E': 38536.0, 'B01003_001E': 659009.0, 'B02001_004E': 2174.0, 'B02001_005E': 24036.0, 'B02001_003E': 318598.0, 'B03001_003E': 69106.0, 'B02001_006E': 271.0, 'B

IndexError: list index out of range